## NLTK documentation advice: 
https://www.nltk.org/api/nltk.lm.html

In [35]:
#install dependencies
from nltk.lm.preprocessing import padded_everygram_pipeline #function used to prepare the tokenized text accordingly
from nltk.lm import MLE
from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends
import string

#install the “popular” subset of NLTK data, on the command line type
#python -m nltk.downloader popular
import nltk.tokenize
import pandas as pd

from nltk.lm import MLE #import a maximum likelihood estimator


In [11]:
#read in data
asap_df = pd.read_excel("../cleanData/processedAsap.xlsx")
test_row = asap_df.iloc[3, :]

In [18]:
#function to tokenize text while keeping stop words
def clean_text_keep_stopword(text):
    sentences = nltk.sent_tokenize(text) #create sentence tokens (not cleaned)
    text = "".join([word for word in text if word not in string.punctuation]) #remove punctuation
    tokens = nltk.tokenize.word_tokenize(text) #tokenize
    return(tokens)



In [19]:
#run the word tokenized function through every row of text
asap_df['word_token_with_stopword'] = asap_df["essay"].apply(lambda x: clean_text_keep_stopword(x.lower())) 

In [24]:
#use padded_everygram_pipeline() to preprocess the tokenized data 
train, vocab = padded_everygram_pipeline(2, asap_df["word_token_with_stopword"]) 

In [26]:
from nltk.lm import MLE #import a maximum likelihood estimator
lm = MLE(2) #use MLE function to create an empty vocabulary

In [27]:
lm.fit(train, vocab) #fit the MLE model to the preprocessed data

In [43]:
#use a list comprehension to run two functions on the word tokens with stopwords for each essay:
 #1) create a series of bigrams based on the text, 2) calculate entropy
asap_df["perplexity"] = asap_df['word_token_with_stopword'].apply(lambda x: lm.perplexity(list(bigrams(pad_both_ends(x, n=2)))))

In [44]:
#check that it worked
asap_df.groupby(["ai_generated"])[["perplexity"]].mean()

,perplexity
ai_generated,
0,83.506328
1,89.592823
